In [4]:
import pandas as pd
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [5]:
last_fy = date.today().year - 1
prev_prev_fy = date.today().year - 2

In [6]:
print(last_fy)

2023


In [7]:
modcols = {
    'Unnamed: 0' : 'metric_type',
    'Unnamed: 1' : f"{last_fy} Totals",
    'Unnamed: 2' : f"Perc of {last_fy}",
    'Unnamed: 3' : f"{prev_prev_fy} Totals",
    'Unnamed: 4' : f"Perc of {prev_prev_fy}"
}

In [8]:
df = pd.read_csv("data/2yrcompare/2023 Profit and Loss-BJW (YTD).csv", index_col = False)

In [9]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,BJ WIN HOSPITALITY INC.,NaN,NaN,NaN,NaN
1,Profit and Loss - 2 Years,NaN,NaN,NaN,NaN
2,"From the Twelve Months Ending December 31, 2023",NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,This Year,Ratio,Last Year,Ratio


In [10]:
df = df.rename(columns = modcols)

In [11]:
df = df.dropna(axis=0)

In [12]:
paranthesis1 = df[f"{last_fy} Totals"].str.contains(r'\(|\)')
paranthesis2 = df[f"{prev_prev_fy} Totals"].str.contains(r'\(|\)')

In [13]:
# Ensure both columsn are first strings in order to clean up using regex
df[f"{last_fy} Totals"] = df[f"{last_fy} Totals"].astype(str)
df[f"{prev_prev_fy} Totals"] = df[f"{prev_prev_fy} Totals"].astype(str)

In [14]:
df.loc[paranthesis1, f"{last_fy} Totals"] = df.loc[paranthesis1, f"{last_fy} Totals"].str.replace(r'\(', '-',regex=True).str.replace(r'\)', '', regex=True)
df[f"{last_fy} Totals"] = df[f"{last_fy} Totals"].str.replace('[\$,]', '', regex=True).astype(float)
df.loc[paranthesis2, f"{prev_prev_fy} Totals"] = df.loc[paranthesis2, f"{prev_prev_fy} Totals"].str.replace(r'\(', '-',regex=True).str.replace(r'\)', '', regex=True)
df[f"{prev_prev_fy} Totals"] = df[f"{prev_prev_fy} Totals"].str.replace('[\$,]', '', regex=True).astype(float)

In [15]:
df.head()

,metric_type,2023 Totals,Perc of 2023,2022 Totals,Perc of 2022
7,MEETING ROOM RENTAL,86694.69,2.03,69576.03,1.42
8,ROOM RENTAL,4155356.65,97.53,4819339.23,98.23
9,MISCELLANEOUS INCOME,188.37,0.00,0.00,0.00
10,REFUND,-58.90,(0.00),-4232.17,(0.09)
11,INTEREST INCOME,18334.51,0.43,19164.56,0.39


In [16]:
df = df.reset_index(drop=True)
# print(df.iloc[7:12])

In [17]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 5,
                       ):
    print(df)


                     metric_type  2023 Totals Perc of 2023  2022 Totals  \
0            MEETING ROOM RENTAL     86694.69         2.03     69576.03   
1                    ROOM RENTAL   4155356.65        97.53   4819339.23   
2           MISCELLANEOUS INCOME       188.37         0.00         0.00   
3                         REFUND       -58.90       (0.00)     -4232.17   
4                INTEREST INCOME     18334.51         0.43     19164.56   
5                DIVIDEND INCOME         0.00         0.00      2215.90   
6                 Total Revenues   4260515.32       100.00   4906063.55   
7               FRONT DESK CLERK    168095.84         3.95    164188.66   
8                    HOUSEKEEPER    412127.92         9.67    474609.95   
9                CAFE ATTENDANTS     89452.20         2.10      1003.00   
10                 PAYROLL TAXES     56329.65         1.32     18203.40   
11              PENSION EXPENSES         0.00         0.00    284463.63   
12        WORKERS COMP IN

In [18]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', 1,
                       'display.precision', 5,
                       ):
    print(df)

                     metric_type  ...
0            MEETING ROOM RENTAL  ...
1                    ROOM RENTAL  ...
2           MISCELLANEOUS INCOME  ...
3                         REFUND  ...
4                INTEREST INCOME  ...
5                DIVIDEND INCOME  ...
6                 Total Revenues  ...
7               FRONT DESK CLERK  ...
8                    HOUSEKEEPER  ...
9                CAFE ATTENDANTS  ...
10                 PAYROLL TAXES  ...
11              PENSION EXPENSES  ...
12        WORKERS COMP INSURANCE  ...
13                LINEN PURCHASE  ...
14                GUEST SUPPLIES  ...
15            GUEST DRY CLEANING  ...
16              LAUNDRY SUPPLIES  ...
17                KITCHEN SUPPLY  ...
18      COMMISSION TRAVEL AGENTS  ...
19         CONTINENTAL BREAKFAST  ...
20           FRANCHISE ROYALTIES  ...
21                    REWARD FEE  ...
22       CREDIT CARD COMMISSIONS  ...
23         MAINTENANCE  SALARIES  ...
24   PAYROLL TAXES - Maintenance  ...
25          

In [19]:
# All modifications of dataframe
df.iloc[10, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES OPS'
df.iloc[45, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES ADMIN'
df.iloc[24, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES MAINT'
df = df.drop(columns=[f"Perc of {last_fy}", f"Perc of {prev_prev_fy}"], axis = 1)
# df = df.drop(columns=[f"Perc of {last_fy}", f"Perc of {prev_prev_fy}"], axis = 1)
# df['Percent Diff'] = np.where(df[f"{prev_prev_fy} Totals"] != 0, round(((df[f"{last_fy} Totals"] - df[f"{prev_prev_fy} Totals"])/df[f"{prev_prev_fy} Totals"])*100, 2), 100)

In [20]:
# print(df.columns)

In [21]:
df['Percent Diff'] = np.where(df[f"{prev_prev_fy} Totals"] != 0, round(((df[f"{last_fy} Totals"] - df[f"{prev_prev_fy} Totals"])/df[f"{prev_prev_fy} Totals"])*100, 2), 100)

In [22]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 6,
                       ):
    print(df)

                     metric_type  2023 Totals  2022 Totals  Percent Diff
0            MEETING ROOM RENTAL     86694.69     69576.03         24.60
1                    ROOM RENTAL   4155356.65   4819339.23        -13.78
2           MISCELLANEOUS INCOME       188.37         0.00        100.00
3                         REFUND       -58.90     -4232.17        -98.61
4                INTEREST INCOME     18334.51     19164.56         -4.33
5                DIVIDEND INCOME         0.00      2215.90       -100.00
6                 Total Revenues   4260515.32   4906063.55        -13.16
7               FRONT DESK CLERK    168095.84    164188.66          2.38
8                    HOUSEKEEPER    412127.92    474609.95        -13.16
9                CAFE ATTENDANTS     89452.20      1003.00       8818.46
10             PAYROLL TAXES OPS     56329.65     18203.40        209.45
11              PENSION EXPENSES         0.00    284463.63       -100.00
12        WORKERS COMP INSURANCE     76154.86     8

#### CALCULATIONS OF CPOR

In [23]:
# columns for cost: totalcost of revenue, officer/admin salaries, payroll taxes, commercial ins, telephone, internet, property tax, 
# credit card commission, maint salary, payroll tax, pest, eqpt rep, fire and safety, swimming pool, maint&serv, plumbing, elec, trash&waste, 
# maint supp, total utilities, auto/gen insurance, building ins, 
rows_to_sum = ['FRONT DESK CLERK', 'HOUSEKEEPER', 'CAFE ATTENDANTS', 'PAYROLL TAXES OPS', 'PENSION EXPENSES', 
               'WORKERS COMP INSURANCE', 'LINEN PURCHASE', 'GUEST SUPPLIES', 'GUEST DRY CLEANING', 'LAUNDRY SUPPLIES', 'KITCHEN SUPPLY',
               'COMMISSION TRAVEL AGENTS', 'CONTINENTAL BREAKFAST', 'FRANCHISE ROYALTIES', 'REWARD FEE', 'CREDIT CARD COMMISSIONS', 'MAINTENANCE  SALARIES', 
               'PAYROLL TAXES MAINT', 'PEST CONTROL', 'MAINTENANCE SUPPLIES', 'EQUIPMENT REPAIR - GENERAL', 'FIRE & SAFETY', 
               'MAINTENANCE & SERV AGRMT', 'SECURITY EXPENSE', 'PLUMBING', 'ELECTRICAL & MECHANICAL', 'ELEVATORS', 'TRASH & WASTE REMOVAL', 'MAINTENANCE SUPPLIES',
               'MISCELLANEOUS - MAINTENANCE', 'WATER', 'GAS', 'ELECTRICITY', 'TEMPORARY EMPLOYEE', 'ENTERTAINMENT', 'TELEPHONE', 'INTERNET & COMPUTER']
df['metric_type'] = df['metric_type'].str.lower().str.strip()
rows_to_sum = [row.lower().strip() for row in rows_to_sum]
temp = df['metric_type'].isin(rows_to_sum)
selectRows = df[temp]
totalOpCostCurrYr = selectRows[f"{last_fy} Totals"].sum()
totalOpCostPrevYr = selectRows[f"{prev_prev_fy} Totals"].sum()

In [24]:
roomsSoldCurrYr = input("How many rooms sold?")
# retrieve total rooms sold from VM or NA reports
# e.g: 2022 total rooms = 35109
# 2023 = 34534
cPORCurrYr = totalOpCostCurrYr/int(roomsSoldCurrYr)
print(f"{last_fy}: ${round(cPORCurrYr, 2)}")

2023: $60.01


In [26]:
roomsSoldPrevYr = input("How many rooms sold?")
# retrieve total rooms sold from VM or NA reports
# e.g: 2021 total rooms = 34811
# 2022 = 35109
cPORPrevYr = totalOpCostPrevYr/int(roomsSoldPrevYr)
print(f"{prev_prev_fy}: ${round(cPORPrevYr, 2)}")

2022: $67.75


#### DISPLAY PERC OF TOTAL COLUMN FOR EXPENSE INSIGHT

In [29]:
totalRevenue = df.loc[6, f"{last_fy} Totals"]

4260515.32


In [37]:
# Group expenses into categories
def categorize_expense(category):
    if (category.upper()) in ['FRONT DESK CLERK', 'HOUSEKEEPER', 'CAFE ATTENDANTS', 'PAYROLL TAXES OPS', 'PENSION EXPENSES', 
                    'WORKERS COMP INSURANCE', 'LINEN PURCHASE', 'GUEST SUPPLIES', 'GUEST DRY CLEANING', 
                    'LAUNDRY SUPPLIES', 'KITCHEN SUPPLY', 'COMMISSION TRAVEL AGENTS', 'CONTINENTAL BREAKFAST', 
                    'FRANCHISE ROYALTIES', 'REWARD FEE', 'CREDIT CARD COMMISSIONS', 'MAINTENANCE  SALARIES', 
                    'PAYROLL TAXES MAINT', 'PEST CONTROL', 'EQUIPMENT REPAIR - GENERAL', 'FIRE & SAFETY', 
                    'SWIMMING POOL', 'MAINTENANCE & SERV AGRMT', 'SECURITY EXPENSE', 'PLUMBING', 
                    'ELECTRICAL & MECHANICAL', 'ELEVATORS', 'LANDSCAPING/EXTERIOR', 'TRASH & WASTE REMOVAL', 'OFFICE SUPPLIES',
                    'MAINTENANCE SUPPLIES', 'MISCELLANEOUS - MAINTENANCE', 'WATER', 'GAS', 'ELECTRICITY','ENTERTAINMENT', 'TELEPHONE', 'INTERNET & COMPUTER']:
        return 'Operational Costs'
    elif (category.upper()) in ['OFFICERS SALARIES', 'ADMINISTRATIVE SALARIES', 'PAYROLL TAXES ADMIN', 'EDUCATION & SEMINAR', 'TEMPORARY EMPLOYEE', 
                      'TRAVEL EXPENSE', 'EMPLOYEE RECRUITMENT', 'PAYROLL SERVICE FEES', 'POSTAGE & DELIVERY',  
                      'MEMBERSHIP DUES/FEES', 'GASOLINE/AUTO EXPENSE', 'ADVERTISING - NEWSPAPER/MAG', 'PROMOTION - GENERAL',
                      'AUTO INSURANCE']:
        return 'Administrative Costs'
    elif (category.upper()) in ['COMMERCIAL INSURANCE', 'LICENSES/PERMITS', 'BANK SERVICE CHARGES', 'MANAGEMENT FEE', 'PROPERTY TAXES', 
                      'LEGAL & PROFESSIONAL FEES', 'DONATION', 'OTHER TAXES', 'BUILDING REPAIRS', 'BUILDING INSURANCE', 'RENOVATION EXPENSES',
                      'INTEREST EXPENSE']:
        return 'Miscellaneous Costs'
    else:
        return 'Revenue'

In [38]:
df['Expense Category'] = df['metric_type'].apply(categorize_expense)

In [71]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 6,
                       ):
    print(df)

                     metric_type  2023 Totals  2022 Totals  Percent Diff  \
0            meeting room rental     86694.69     69576.03         24.60   
1                    room rental   4155356.65   4819339.23        -13.78   
2           miscellaneous income       188.37         0.00        100.00   
3                         refund       -58.90     -4232.17        -98.61   
4                interest income     18334.51     19164.56         -4.33   
5                dividend income         0.00      2215.90       -100.00   
6                 total revenues   4260515.32   4906063.55        -13.16   
7               front desk clerk    168095.84    164188.66          2.38   
8                    housekeeper    412127.92    474609.95        -13.16   
9                cafe attendants     89452.20      1003.00       8818.46   
10             payroll taxes ops     56329.65     18203.40        209.45   
11              pension expenses         0.00    284463.63       -100.00   
12        wo

In [55]:
# Calculate total for each expense category
# expense_totals = df[df['Expense Category'] != 'Revenue'].groupby('Expense Category')[f"{last_fy} Totals"].sum()

# Calculate percentages
df['expense_percentages'] = (df[f"{last_fy} Totals"] / totalRevenue) * 100
df.head()

,metric_type,2023 Totals,2022 Totals,Percent Diff,Expense Category,expense_percentages
0,meeting room rental,86694.69,69576.03,24.60,Revenue,2.034840
1,room rental,4155356.65,4819339.23,-13.78,Revenue,97.531785
2,miscellaneous income,188.37,0.00,100.00,Revenue,0.004421
3,refund,-58.90,-4232.17,-98.61,Revenue,-0.001382
4,interest income,18334.51,19164.56,-4.33,Revenue,0.430336


In [68]:
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
from IPython.display import display

In [70]:
operationsSlice = df[df['Expense Category'] == 'Operational Costs']
adminSlice = df[df['Expense Category'] == 'Administrative Costs']
miscSlice = df[df['Expense Category'] == 'Miscellaneous Costs']

titles = ['Ops Perc Total', 'Admin Perc Total', 'Misc Perc Total']

fig = make_subplots(rows=1,cols=3, subplot_titles=titles, specs=[[{"type":"domain"} for _ in range(3)] for _ in range(1)], print_grid=True)  

for i, data_slice in enumerate([operationsSlice, adminSlice, miscSlice]):
    label = data_slice['metric_type']
    size = data_slice['expense_percentages']
    fig_pie=go.Pie(values=size, labels=label, showlegend=False, textposition='inside', textinfo='percent', hoverinfo='label+percent')
    fig.add_trace(fig_pie, row=1, col=i+1)

display(fig)

This is the format of your plot grid:
[ (1,1)  ]  [ (1,2)  ]  [ (1,3)  ]



### Expense Ratio

In [72]:
df['expense_ratio'] = 0
df.loc[7:78, 'expense_ratio'] = df.loc[7:78, 'Percent Diff']/df.loc[6, 'Percent Diff']
print(df)

                     metric_type  2023 Totals  2022 Totals  Percent Diff  \
0            meeting room rental     86694.69     69576.03         24.60   
1                    room rental   4155356.65   4819339.23        -13.78   
2           miscellaneous income       188.37         0.00        100.00   
3                         refund       -58.90     -4232.17        -98.61   
4                interest income     18334.51     19164.56         -4.33   
..                           ...          ...          ...           ...   
76                       penalty         0.00      3363.98       -100.00   
77  provision for in tax - state         0.00      8990.00       -100.00   
78    provision for in tax - pte         0.00     55738.00       -100.00   
79                total expenses   1741773.37   2249771.54        -22.58   
80                    net income    546087.60    390201.12         39.95   

   Expense Category  expense_percentages  expense_ratio  
0           Revenue          

/tmp/ipykernel_236321/1013376701.py:2: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-1.80851064e-01  1.00000000e+00 -6.70095745e+02 -1.59156535e+01
  7.59878419e+00  4.72644377e-01 -4.63525836e-01 -2.55319149e-01
 -3.88951368e+01  1.38677812e+00 -7.59878419e+00  6.91489362e-02
  5.31914894e-03 -3.13829787e-01  5.28875380e-01  6.71732523e-01
  5.07598784e-01  6.86170213e+00 -7.45440729e-01  1.02431611e+00
  7.08054711e+00 -1.41869301e+00 -1.49164134e+00 -2.31534954e+00
  6.98480243e+00  7.36474164e+00  1.70364742e+00 -1.22264438e+01
 -2.61398176e-01  3.05547112e+00 -7.59878419e+00  2.24924012e+00
 -1.10790274e+00 -7.52279635e-01  9.84042553e-01  1.01367781e+00
 -0.00000000e+00  2.86474164e-01  2.14285714e-01 -5.45357143e+01
  3.39361702e+00 -7.76823708e+00  1.36246201e+00 -7.59878419e+00
 -1.72788754e+01  1.72644377e+00 -4.96200608e-01 -2.84194529e-01
  7.12993921e+00 -7.59878419e+00  7.59878419e+00 -8

In [75]:
cost_ops = df[df['Expense Category'] == 'Operational Costs'].copy()
admin_exp = df[df['Expense Category'] == 'Administrative Costs'].copy()
miscellaneous_exp = df[df['Expense Category'] == 'Miscellaneous Costs'].copy()

cost_ops['category'] = 'Cost Ops'
admin_exp['category'] = 'Admin Expense'
miscellaneous_exp['category'] = 'Maint Expense'

combined_df = pd.concat([cost_ops, admin_exp, miscellaneous_exp], ignore_index=True)

combined_df = combined_df[combined_df['Percent Diff'].abs() != 100]

bar = px.bar(combined_df, x='metric_type', y='expense_ratio', color = 'category', labels={'Expense Efficiency Ratio': 'Expense Efficiency Ratio'})
bar.update_yaxes(type='log')
bar.update_layout(
    title='Expense Efficiency Ratio by Metric Type',
    xaxis_title='Metric Type',
    yaxis_title='Expense Efficiency Ratio'
)
bar.add_annotation(
    text="Values > 1 indicate expense grew YoY compared to revenue for this metric",
    xref="paper", yref="paper",
    x=1, y=1.1,
    showarrow=False,
    font=dict(size=12)
)
bar.show()                                                             